## Sacar información de la película

In [1]:
import urllib.request
import re

In [2]:
CLASS_DURACION = 'ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt'  # Tiene dentro 3 li con año, clasificación y duración
CLASS_RATING = 'sc-d541859f-1 imUuxf'  # Dentro de un span
CLASS_NUMEROVOTOS = 'sc-d541859f-3 dwhNqC' # Dentro de un div
CLASS_SINOPSIS = 'sc-42125d72-1 igbBrx'
# Coger los 3 primeros (direccion, guionistas, elenco)
CLASS_DATOS = 'ipc-metadata-list-item__content-container'  # Dentro de un div



# Definir la URL
url = 'https://www.imdb.com/es/title/tt27722575/?ref_=nv_sr_srsg_0_tt_8_nm_0_in_0_q_200'

# Definir los headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Crear una petición con los headers
req = urllib.request.Request(url, headers=headers)

# Descargar el HTML
with urllib.request.urlopen(req) as response:
    html = response.read().decode('utf-8')

# Imprimir el HTML descargado
print(response.status)

200


In [3]:
# Sacamos (Año), (Calificación), (Duración)
extractor = rf'.*\<ul class="{CLASS_DURACION}"[^\>]*?\>\<li[^\>]*?\>\<a[^\>]*?\>([\d]+?)\<.+?\<li[^\>]*?\>\<a[^\>]*?\>([^<]+?)\<.+?\<li[^\>]*?\>([^\<]+?)\<.+' 
# Sacamos (puntuación)
extractor += rf'\<span class="{CLASS_RATING}"\>([\d\.]+)\<.+'
# Sacamos (nº de votos)
extractor += rf'\<div class="{CLASS_NUMEROVOTOS}"\>([\d]+?)\<.+'
# Sacamos (sinopsis)
extractor += rf'\<span [^\>]*?class="{CLASS_SINOPSIS}"\>([^<]+?)\<'
resultados =  re.search(extractor, html, re.DOTALL)
anio, calificacion, duracion, puntuacion, num_votos, sinopsis = resultados.groups()
print(anio, calificacion, duracion, puntuacion, num_votos, sinopsis)

2024 PG 1h 38min 5.7 911 Un heroico perro poodle que lucha por mantener el respeto de su manada de hombres lobo. Cuando un deseo caprichoso lo transforma en hombre lobo y despoja a un hada traviesa de la tierra.


In [4]:
# Sacamos (direccion), (guionistas), (elenco) con un findall
informacion = rf'\<(?:span|a).*?aria-label="Ver elenco y equipo completos".+?\>([^\>]+?)\<\/(?:span|a)\>\<div class="{CLASS_DATOS}"\>\<ul.+?\>(.+?)\<\/ul\>'
# Limpiamos los resultados sacando los nombres y quitando las etiquetas HTML
obtener_nombres = re.compile(r'([^\>]+)\<\/a\>')

In [5]:
# Encontrar todas las coincidencias
coincidencias = re.finditer(informacion, html)

# Crear una lista para almacenar los primeros 4 resultados
resultados = []

# Iterar sobre las coincidencias y agregar a la lista hasta encontrar 4
for i, match in enumerate(coincidencias):
    if i >= 3:
        break
    resultados.append((match.group(1), obtener_nombres.findall(match.group(2))))

for i, match in enumerate(resultados):
    print(f"Resultado {i}: {match}")

Resultado 0: ('Dirección', ['Alexs Stadermann'])
Resultado 1: ('Guionistas', ['Fin Edquist', 'Jayne Lyons'])
Resultado 2: ('Elenco', ['Ilai Swindells', 'Elizabeth Nabben', 'Jennifer Saunders'])
